### --- Day 22: Sand Slabs ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/22

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 22);
WriteLines(inputLines);

Loading puzzle file: Day22.txt
Total lines: 1232
Max line length: 15

6,8,103~7,8,103
1,4,214~1,4,216
1,4,10~1,5,10
4,5,56~4,9,56
7,5,109~8,5,109


In [4]:
record Point(int x, int y, int z);

record Brick(Point start, Point end) 
{
    public HashSet<Point> AllPoints 
    {
        get 
        {
            HashSet<Point> result = new();

            for (var x = start.x; x <= end.x; x++)
            for (var y = start.y; y <= end.y; y++)
            for (var z = start.z; z <= end.z; z++) 
            {
                result.Add(new Point(x, y, z));
            }

            return result;
        }
    }

    public IEnumerable<Point> BottomPoints
    {
        get 
        {
            for (var x = start.x; x <= end.x; x++)
            for (var y = start.y; y <= end.y; y++) 
            {
                yield return new Point(x, y, start.z);
            }
        }
    }

    public Brick MoveDown(int fallDistance) => new Brick(new Point(start.x, start.y, start.z - fallDistance), new Point(end.x, end.y, end.z - fallDistance));
}

static Brick ParseBrick(string line) {
    var bits = line.Split('~')
        .SelectMany(x => x.Split(','))
        .Select(int.Parse)
        .ToArray();

    var brick = bits switch {
        [var a, var b, var c, var d, var e, var f] => new Brick(new Point(a, b, c), new Point(d, e, f)),
        _ => throw new ArgumentException($"Unable to parse line {line}")
    };

    return brick;
}

In [5]:
string[] testInputLines = [
    "1,0,1~1,2,1",
    "0,0,2~2,0,2",
    "0,2,3~2,2,3",
    "0,0,4~0,2,4",
    "2,0,5~2,2,5",
    "0,1,6~2,1,6",
    "1,1,8~1,1,9",
];

var testInputBricks = testInputLines.Select(ParseBrick).ToArray();

// foreach (var b in testInputBricks) {
//     Console.WriteLine(string.Join(", ", b.AllPoints));
// }

foreach (var bp in testInputBricks[0].BottomPoints) {
    Console.WriteLine(bp);
}

Point { x = 1, y = 0, z = 1 }
Point { x = 1, y = 1, z = 1 }
Point { x = 1, y = 2, z = 1 }


In [6]:
class DropSim 
{
    readonly Dictionary<Brick, bool> loadBearing = new();
    readonly Dictionary<Point, Brick> brickPoints = new();

    // needed for part 2
    readonly Dictionary<Brick, HashSet<Brick>> supporting = new();
    readonly Dictionary<Brick, HashSet<Brick>> supportedBy = new();

    public void Drop(Brick brick) {
        // Get the bottom points of the brick
        var bottomPoints = brick.BottomPoints.ToList();

        // Move them down until at least one pixel is occupied.
        var zStart = brick.start.z;
        var zLand = zStart - 1;
        HashSet<Brick> collisions = new();
        while (zLand >= 0)
        {
            var q = from bp in bottomPoints
                    let newPoint = new Point(bp.x, bp.y, zLand)
                    where brickPoints.ContainsKey(newPoint)
                    select brickPoints[newPoint];

            collisions.UnionWith(q);

            if (collisions.Count > 0) {
                // The resting point is 1 above this point
                break;
            }
            zLand--;
        }
        zLand += 1;

        // The brick(s) underneath are now load-bearing
        var fallDistance = zStart - zLand;
        var landedBrick = brick.MoveDown(fallDistance);

        loadBearing[landedBrick] = false;
        foreach (var p in landedBrick.AllPoints) {
            brickPoints[p] = landedBrick;
        }

        if (collisions.Count == 1)
        {
            // Only one block holds up this falling block. Therefore the
            // supporting block is a load-bearing block
            
            loadBearing[collisions.Single()] = true;
        }

        // Part 2
        supporting[landedBrick] = new();
        supportedBy[landedBrick] = new();
        foreach (var c in collisions) 
        {
            supporting[c].Add(landedBrick);
            supportedBy[landedBrick].Add(c);
        }
    }

    public string Render() {
        var sb = new StringBuilder();
        
        foreach (var lb in loadBearing) {
            var brick = lb.Key;
            sb.AppendLine($"{brick.start} -> {brick.end} ({lb.Value})");
        }

        return sb.ToString();
    }

    // public int SafeToDisintegrate => loadBearing.Where(lb => !lb.Value).Count();

    public int SafeToDisintegrate 
    {
        get {
            // This is a convoluted way, just to test our supporting / supportedBy
            var loadBearing = supporting
                // We are supporting blocks
                .Where(s => s.Value.Count > 0)
                // Those blocks are only supported by us
                .Where(s => s.Value.Any(ss => supportedBy[ss].Count == 1))
                .Count();

            return supporting.Count - loadBearing;
        }
    }

    public IEnumerable<Brick> AllBricks => loadBearing.Keys;
    public IEnumerable<Brick> LoadBearing => loadBearing.Where(lb => lb.Value).Select(lb => lb.Key);
    public Dictionary<Brick, HashSet<Brick>> Supporting => supporting;
    public Dictionary<Brick, HashSet<Brick>> SupportedBy => supportedBy;
}

var testSim = new DropSim();
// testSim.Drop(testInputBricks[0]);
// testSim.Drop(testInputBricks[1]);
foreach (var b in testInputBricks) {
    testSim.Drop(b);
}

Console.WriteLine(testSim.Render());

// So, in this example, 5 bricks can be safely disintegrated.
Console.WriteLine(testSim.SafeToDisintegrate);

Point { x = 1, y = 0, z = 0 } -> Point { x = 1, y = 2, z = 0 } (True)
Point { x = 0, y = 0, z = 1 } -> Point { x = 2, y = 0, z = 1 } (False)
Point { x = 0, y = 2, z = 1 } -> Point { x = 2, y = 2, z = 1 } (False)
Point { x = 0, y = 0, z = 2 } -> Point { x = 0, y = 2, z = 2 } (False)
Point { x = 2, y = 0, z = 2 } -> Point { x = 2, y = 2, z = 2 } (False)
Point { x = 0, y = 1, z = 3 } -> Point { x = 2, y = 1, z = 3 } (True)
Point { x = 1, y = 1, z = 4 } -> Point { x = 1, y = 1, z = 5 } (False)

5


In [7]:
var inputBricks = inputLines.Select(ParseBrick).OrderBy(b => b.start.z).ToArray();

var part1Sim = new DropSim();
foreach (var b in inputBricks) {
    part1Sim.Drop(b);
}

In [8]:
// Figure how the blocks will settle based on the snapshot. Once they've
// settled, consider disintegrating a single brick; how many bricks could be safely
// chosen as the one to get disintegrated?

var part1Answer = part1Sim.SafeToDisintegrate;
Console.WriteLine(part1Answer);

430


In [9]:
// 430 is correct!
Ensure(430, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/22

In [11]:
// Ok, so trying to disintegrate a non-load-bearing brick will do nothing, since there will be another brick holding up anything above. So the load-bearing bricks are the roots of our search.

// Imagine this case:

        //    *
        //    *           F
        // *********      E
        //   *    *
        //   *    *       C     D
        // **********     B
        //     *
        //     *          A
        //     *

// A is load-bearing because it is supporting B
// B is load-bearing because it is supporting C and D
// C and D are NOT load bearing because E is supported by both
// E is load-bearing because it is supporting F
// F is NOT load-bearing because it is not supporting anything
//
// ...but removing A would cause a chain-reaction that would cause all of them to fall

// Potentially, we can consider each load-bearing brick as as root, and trace upwards. 

// For each brick, we mark all the bricks it's supporting

// For those bricks, if they are only supported by marked bricks, mark them and trace upwards

// If they are supported by a non-marked brick, they will not fall and neither will any above

In [12]:
record TracedBrick(HashSet<Brick> coloured, HashSet<Brick> supporting)
{
    public int ResultCount => coloured.Count - 1;
}

class BrickTracer 
{
    Dictionary<Brick, TracedBrick> traced = new();

    public TracedBrick TraceOne(DropSim sim, Brick brickRoot)
    {
        Queue<Brick> queue = new();
        HashSet<Brick> coloured = new();
        HashSet<Brick> supportingBricks = new();

        queue.Enqueue(brickRoot);

        while (queue.TryDequeue(out var currentBrick))
        {
            var supportedBy = sim.SupportedBy[currentBrick];
            var inColour = supportedBy.All(sb => coloured.Contains(sb));
            if (!inColour && currentBrick != brickRoot) {
                // This brick is supported by bricks outside our trace. It will not fall.
                
                // (From a caching perspective, we want to know how many bricks
                // will fall, but also which ones remain supported, as future
                // traces further down the stack might cause this whole set to fall.)
                supportingBricks.Add(currentBrick);
                continue;
            }

            // This brick is in our trace. Add it to the set and consider the bricks
            // it's supporting 

            coloured.Add(currentBrick);
            var supporting = sim.Supporting[currentBrick];

            if (traced.TryGetValue(currentBrick, out var existing)) {
                // This is a load-bearing brick that we have previously traced. We
                // can save re-tracing the internals of this brick.

                coloured.UnionWith(existing.coloured);
                supporting = existing.supporting;
            }

            foreach (var s in supporting)
            {
                queue.Enqueue(s);
            }
        }

        TracedBrick result = new(coloured, supportingBricks);
        traced[brickRoot] = result;

        return result;
    }
}

In [13]:
// Using the same example as above:

// Disintegrating brick A would cause all 6 other bricks to fall.
// Disintegrating brick F would cause only 1 other brick, G, to fall.
// Disintegrating any other brick would cause no other bricks to fall. So, in
// this example, the sum of the number of other bricks that would fall as a result
// of disintegrating each brick is 7.

var testTracer = new BrickTracer();
foreach (var s in testSim.LoadBearing)
{
    var xx = testTracer.TraceOne(testSim, s);

    Console.WriteLine($"Checking {s.start} = {xx.ResultCount} additional bricks");
}

Checking Point { x = 1, y = 0, z = 0 } = 6 additional bricks
Checking Point { x = 0, y = 1, z = 3 } = 1 additional bricks


In [14]:
// For each brick, determine how many other bricks would fall if that brick were
// disintegrated. What is the sum of the number of other bricks that would fall?

var part2Tracer = new BrickTracer();
var part2Answer = part1Sim.LoadBearing.Reverse().Select(lb => part2Tracer.TraceOne(part1Sim, lb).ResultCount).Sum();
Console.WriteLine(part2Answer);

60558


In [15]:
// 60558 is correct!
Ensure(60558, part2Answer);